# FUNCIONES API

In [1]:
import pandas as pd
import json
import numpy as np

Función normalización de generos

In [2]:
def get_genre(genre_input: str):
    # Lista de géneros
    genres = ["Action", "Indie", "Adventure", "RPG", "Strategy", "Free to Play", 
              "Simulation", "Casual", "Massively Multiplayer", "Early Access", 
              "Sports", "Racing", "Utilities", "Design & Illustration", 
              "Animation & Modeling", "Video Production", "Web Publishing", 
              "Education", "Software Training", "Audio Production", "Photo Editing"]

    # Convierte el género de entrada y los géneros de la lista a minúsculas para la comparación
    genre_input_lower = genre_input.lower()
    genres_lower = [genre.lower() for genre in genres]

    # Comprueba si el género de entrada está en la lista de géneros
    if genre_input_lower in genres_lower:
        # Si está en la lista, devuelve el género tal cual aparece en la lista original
        return genres[genres_lower.index(genre_input_lower)]
    else:
        return None


In [3]:
# Prueba la función
print(get_genre("ACTION"))
print(get_genre("RPG"))
print(get_genre("free to play"))
print(get_genre("unknown genre"))

Action
RPG
Free to Play
None


## PlayTimeGenre( genero : str ): 
Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [4]:
def PlayTimeGenre(genero : str):
    genero = get_genre(genero)
    if genero == None:
        return {"Error": "El género no existe"}
    else:
        ranking_genre = pd.read_parquet('../dataset/ranking_genre.parquet')
        filtro = ranking_genre[ranking_genre['genres'] == genero]
        del ranking_genre
        return {f"Año de lanzamiento con más horas jugadas para Género {genero}" : int(filtro['year'].iloc[0])}

In [5]:
PlayTimeGenre('s')

{'Error': 'El género no existe'}

## def UserForGenre( genero : str ):
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [6]:
def UserForGenre(genero : str):
    genero = get_genre(genero)
    if genero == None:
        return {"Error": "El género no existe"}
    else:
        usersXgenre = pd.read_parquet('../dataset/user_genre.parquet')
        filtro = usersXgenre[usersXgenre['genres'] == genero]
        filtro_usuario = filtro.groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False)
        resultado = filtro[filtro['user_id'] == filtro_usuario.index[0]].sort_values('year', ascending=False)
        i = 0
        lista = []
        while i < len(resultado):
            lista.append({f"Año:{int(resultado['year'].iloc[i])}, Horas:{int(resultado['playtime_forever'].iloc[i])}"})
            i += 1
        del usersXgenre
        return {f"Usuario con más horas jugadas para Género {resultado['genres'].iloc[0]}: {resultado['user_id'].iloc[0]},{lista}"}

In [7]:
UserForGenre('PG')

{'Error': 'El género no existe'}

## def UsersRecommend( año : int ): 
Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [8]:
def UsersRecommend(año : int):
    recommend = pd.read_parquet('../dataset/recommend.parquet')
    filtro = recommend[recommend['year'] == año].sort_values(by=['count'], ascending=False).head(3)
    del recommend
    if len(filtro) == 0:
        return {f"No hay juegos recomendados para el año {año}"}
    else:
        i = 0
        lista = []
        while i < len(filtro):
            lista.append({f"Puesto {i+1}" : filtro['title'].iloc[i]})
            i += 1
        return lista

In [9]:
UsersRecommend(2015)

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2': 'Team Fortress 2'},
 {'Puesto 3': "Garry's Mod"}]

## def UsersNotRecommend( año : int ): 
Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [10]:
def UsersNotRecommend( año : int ):
    notrecommend = pd.read_parquet('../dataset/notrecommend.parquet')
    filtro = notrecommend[notrecommend['year'] == año].sort_values(by=['year'], ascending=False).head(3)
    del notrecommend
    if len(filtro) == 0:
        return {f"No hay registros de juegos con peores recomendaciones para el año {año}"}
    else:
        lista = []
        for i in range(0,3):
            lista.append({f"Puesto {i+1}" : filtro['title'].iloc[i]})
        return lista

In [11]:
UsersNotRecommend(2010)

[{'Puesto 1': 'Amnesia: The Dark Descent'},
 {'Puesto 2': 'Empire: Total War™'},
 {'Puesto 3': 'Killing Floor'}]

## def sentiment_analysis( año : int ):
Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [12]:
def sentiment_analysis( año : int ):
    sentiments  = pd.read_parquet('..\dataset\sentiments.parquet')
    filtro = sentiments[sentiments['release_date'].dt.year == año]
    filtro['sentiment_analysis'] = filtro['sentiment_analysis'].astype(int)
    filtro['release_date'] = filtro['release_date'].astype('int64').astype('int32')
    resumen = filtro.groupby(filtro['sentiment_analysis'])['release_date'].count().to_dict()
    resultado = f"Negative={resumen.get(0, 0)}, Neutral={resumen.get(1, 0)}, Positive={resumen.get(2, 0)}"
    del sentiments
    return {resultado}

In [13]:
sentiment_analysis(2011)

C:\Users\stdio\AppData\Local\Temp\ipykernel_14296\1652367928.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro['sentiment_analysis'] = filtro['sentiment_analysis'].astype(int)
C:\Users\stdio\AppData\Local\Temp\ipykernel_14296\1652367928.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro['release_date'] = filtro['release_date'].astype('int64').astype('int32')


{'Negative=471, Neutral=721, Positive=2255'}

# Modelo de aprendizaje automático
## def recomendacion_juego( id de producto ):
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [14]:
#Funcion cosine_similarity
def similarityCosine(vector1, vector2):
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    if norm1 == 0 or norm2 == 0:
        return 0 
    else:
        return np.dot(vector1, vector2) / (norm1 * norm2)


In [15]:
def recomendacion_juego(id : int):
    reviews  = pd.read_parquet('../dataset/reviews_per_item.parquet')
    filtro = reviews[reviews['id'] == id]
    try:
        fila1 = filtro.iloc[0]
    except IndexError:
        return {"error": "No se encontraron datos para el ID del juego dado."}   
    fila1 = filtro.iloc[0]
    lista = []
    i = 0
    while i < len(reviews):
        fila2 = reviews.iloc[i]
        lista.append(similarityCosine(fila1,fila2))
        i += 1
    reviews['similarity_cosine'] = lista
    recomendacion =reviews[['id','similarity_cosine']].sort_values(by=['similarity_cosine'], ascending=False).head(10)
    del reviews
    rec = []
    j = 0
    while j < 6:
        if recomendacion['id'].iloc[j] != id:
            rec.append(recomendacion['id'].iloc[j])
        j+=1
    return {f"Los juegos recomendados para {id} son: {rec}"}

In [16]:
recomendacion_juego(0)

{'error': 'No se encontraron datos para el ID del juego dado.'}